In [ ]:
import pandas as pd
import numpy as np
import sklearn

# コピペ用の関数たち

In [130]:
def merge_train_data(files:list)->pd.DataFrame:
    count = 0
    for file in files:
        train_i = pd.read_csv(file)
        if (count==0):
            df = train_i.copy()
            count+=1
            continue
        df = pd.concat([df,train_i],axis = 0)
    return df
df = merge_train_data(files)

def data_extract(df:pd.DataFrame)->pd.DataFrame:
    # 確実にいらないカラムたち
    data = df.copy()
    data = data.drop(['ID','種類','地域', '都道府県名', '市区町村名', '地区名','土地の形状', '間口', 
                                  '延床面積（㎡）','用途','今後の利用目的','前面道路：方位', '前面道路：種類', 
                                  '前面道路：幅員（ｍ）','取引の事情等','最寄駅：名称'],axis=1) 
    # もしかしたら使うかもしれないカラムたち
    data = data.drop(['市区町村コード','都市計画'],axis=1)
    data = data.fillna({'改装':'未改装'})
    data.rename(columns={'最寄駅：距離（分）':'nearest_station','間取り':'floor_type','面積（㎡）':'area','建築年':'builted_year',
            '建ぺい率（％）':'coverrage','容積率（％）':'floor_rate','取引時点':'sold_year','建物の構造':'structure'},inplace = True)
    return data

def convert_nearest_station(value):
    if value == value:
        if value == '30分?60分':
            return 45
        elif value ==  '1H?1H30':
            return 75
        elif value =='1H30?2H':
            return 105
        elif value =='2H?':
            return 120
        return float(value)
    
def convert_area(area):
    if (area=="2000㎡以上"):
        return 2000
    else:
        return int(area)
    
def convert_builted_year(year):
    if year == year:
        wareki = year[:2]
        if(wareki=="戦前"):
            return 1945
        
        if(year[2:len(year)-1]=="元"):
            value =  1
        else:
            value = int(year[2:len(year)-1])

        if(wareki == "昭和"):
            return 1925 + value
        elif(wareki== "平成"):
            return 1988 +value
        elif(wareki=="令和"):
            return 2018 + value

def convert_sold_year(year):
    return int(year[:4])
    
            
def data_processing(data:pd.DataFrame)->pd.DataFrame:
    df = data.copy()
    df.nearest_station = df.nearest_station.apply(lambda x: convert_nearest_station(x))
    df.builted_year = df.builted_year.apply(lambda x:convert_builted_year(x))
    df.area = df.area.apply(lambda x:convert_area(x))
    df.sold_year = df.sold_year.apply(lambda x:convert_sold_year(x))
    
    dummies = pd.get_dummies(data[['改装']])
    df = pd.concat([df.drop('改装',axis=1),dummies[['改装_改装済']]],axis = 1)
    
    df = df.drop(["floor_type","structure"],axis=1)
    return df
    
    


In [34]:
data = pd.read_csv("./data/train/01.csv")
data.head(1)

,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,1000809,中古マンション等,NaN,1101,北海道,札幌市中央区,北１条西,西１８丁目,5,オープンフロア,...,NaN,NaN,NaN,商業地域,80.0,400.0,2019年第3四半期,改装済,NaN,6.963788


# 削除したカラム

・ID:不要 \
・種類:全部同じ値のため　　\
・地域:市区町村コードで代用できると判断したため\
・都道府県名:"地域"と同様の理由\
・地区名:"地域"と同様の理由 \
・土地の形状:ほぼNULL値のため不要\
・間口:ほぼNULL値のため不要 \
・延床面積（㎡）:ほぼNULL値のため不要 \
・用途:多くの値が「住宅」を占めるため不要と判断\
・今後の利用目的:多くの値が「住宅」を占めるため不要と判断 
#### "用途"と"今後の利用目的"結合すればNULL値がなくなりそうなので要検討
・前面道路：方位:ほぼNULL値のため不要 \
・前面道路：種類:ほぼNULL値のため不要 \
・前面道路：幅員（ｍ）:ほぼNULL値のため不要\
・取引の事情等:ほぼNULL値のため不要

In [35]:
data = pd.read_csv("./data/train/01.csv").head(10)
# 確実にいらないカラムたち
data = data.drop(['ID','種類','地域', '都道府県名', '市区町村名', '地区名','土地の形状', '間口', 
                                  '延床面積（㎡）','用途','今後の利用目的','前面道路：方位', '前面道路：種類', 
                                  '前面道路：幅員（ｍ）','取引の事情等','最寄駅：名称'],axis=1) 
# もしかしたら使うかもしれないカラムたち
data = data.drop(['市区町村コード','都市計画'],axis=1)
# data = data.fillna({'改装':'未改装'})
data.rename(columns={'最寄駅：距離（分）':'nearest_station','間取り':'floor_type','面積（㎡）':'area','建築年':'builted_year',
            '建ぺい率（％）':'coverrage','容積率（％）':'floor_rate','取引時点':'sold_year','建物の構造':'structure'},inplace = True)
data

,nearest_station,floor_type,area,builted_year,structure,coverrage,floor_rate,sold_year,改装,取引価格（総額）_log
0,5,オープンフロア,85,昭和49年,ＳＲＣ,80.0,400.0,2019年第3四半期,改装済,6.963788
1,10,３ＬＤＫ,70,平成7年,ＲＣ,60.0,200.0,2013年第1四半期,未改装,7.000000
2,5,３ＬＤＫ,70,平成6年,ＲＣ,60.0,200.0,2019年第4四半期,改装済,7.301030
3,4,３ＬＤＫ,85,平成15年,ＳＲＣ,60.0,300.0,2009年第2四半期,未改装,7.255273
4,3,１Ｋ,20,昭和64年,ＳＲＣ,80.0,600.0,2017年第1四半期,未改装,6.361728
5,6,３ＬＤＫ,60,昭和52年,ＲＣ,60.0,200.0,2019年第3四半期,未改装,6.740363
6,9,３ＬＤＫ,85,平成20年,ＲＣ,60.0,200.0,2011年第1四半期,未改装,7.361728
7,8,３ＬＤＫ,55,昭和52年,ＲＣ,60.0,200.0,2016年第1四半期,NaN,6.681241
8,4,４ＬＤＫ,90,平成9年,ＳＲＣ,80.0,300.0,2008年第1四半期,未改装,7.301030
9,10,３ＬＤＫ,75,平成19年,ＲＣ,60.0,200.0,2015年第4四半期,未改装,7.397940


### 間取りを順序尺度に変換できれば使えそう

・面積の平均値によってランクを付与したものを使用する\
・部屋数と間取りを対応させる
・

In [110]:
def merge_train_data(files:list)->pd.DataFrame:
    count = 0
    for file in files:
        train_i = pd.read_csv(file)
        if (count==0):
            df = train_i.copy()
            count+=1
            continue
        df = pd.concat([df,train_i],axis = 0)
    return df
df = merge_train_data(files)

def data_extract(df:pd.DataFrame)->pd.DataFrame:
    # 確実にいらないカラムたち
    data = df.copy()
    data = data.drop(['ID','種類','地域', '都道府県名', '市区町村名', '地区名','土地の形状', '間口', 
                                  '延床面積（㎡）','用途','今後の利用目的','前面道路：方位', '前面道路：種類', 
                                  '前面道路：幅員（ｍ）','取引の事情等','最寄駅：名称'],axis=1) 
    # もしかしたら使うかもしれないカラムたち
    data = data.drop(['市区町村コード','都市計画'],axis=1)
    data = data.fillna({'改装':'未改装'})
    data.rename(columns={'最寄駅：距離（分）':'nearest_station','間取り':'floor_type','面積（㎡）':'area','建築年':'builted_year',
            '建ぺい率（％）':'coverrage','容積率（％）':'floor_rate','取引時点':'sold_year','建物の構造':'structure'},inplace = True)
    return data

/Users/komachiyuuki/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [41]:
data['nearest_station'].unique()

array(['7', '4', '18', '5', '6', '22', nan, '8', '16', '3', '30分?60分',
       '29', '9', '17', '12', '23', '14', '21', '15', '25', '20', '19',
       '10', '13', '11', '1', '24', '1H30?2H', '28', '26', '2', '1H?1H30',
       '27', '0', '2H?'], dtype=object)

# nearest_station の前処理

In [111]:
data = data_extract(df)
data.head()

,nearest_station,floor_type,area,builted_year,structure,coverrage,floor_rate,sold_year,改装,取引価格（総額）_log
0,7,２ＬＤＫ,45,昭和63年,ＲＣ,60.0,150.0,2021年第2四半期,改装済,7.041393
1,4,１Ｋ,15,昭和64年,ＳＲＣ,80.0,400.0,2020年第2四半期,未改装,6.602060
2,18,４ＬＤＫ,80,昭和63年,ＲＣ,60.0,150.0,2020年第4四半期,改装済,7.397940
3,5,２ＬＤＫ,60,平成18年,ＲＣ,80.0,400.0,2020年第1四半期,未改装,7.278754
4,6,３ＤＫ,60,昭和62年,ＳＲＣ,80.0,400.0,2007年第3四半期,未改装,6.929419


In [42]:
def convert_nearest_station(value):
    if value == value:
        if value == '30分?60分':
            return 45
        elif value ==  '1H?1H30':
            return 75
        elif value =='1H30?2H':
            return 105
        elif value =='2H?':
            return 120
        return float(value)

In [45]:
data['nearest_station'] = data['nearest_station'].apply(lambda x: convert_nearest_station(x))

In [46]:
moyori =  data.query('nearest_station=="3"').iloc[3,6]
moyorib =  data.query('nearest_station=="3"').iloc[4,6]
print(moyori == moyori)
print(moyorib == moyorib)

False
True


In [48]:
test = data.copy()
station_mean = test.dropna()["nearest_station"].median()
test.fillna({"nearest_station":station_mean},inplace=True)
test.nearest_station.unique()

array([  7.,   4.,  18.,   5.,   6.,  22.,   8.,  16.,   3.,  45.,  29.,
         9.,  17.,  12.,  23.,  14.,  21.,  15.,  25.,  20.,  19.,  10.,
        13.,  11.,   1.,  24., 105.,  28.,  26.,   2.,  75.,  27.,   0.,
       120.])

# builted_year の前処理

そのままだと和暦になっているので西暦にして返す

In [113]:
def convert_builted_year(year):
    if year == year:
        wareki = year[:2]
        if(wareki=="戦前"):
            return 1945
        
        if(year[2:len(year)-1]=="元"):
            value =  1
        else:
            value = int(year[2:len(year)-1])

        if(wareki == "昭和"):
            return 1925 + value
        elif(wareki== "平成"):
            return 1988 +value
        elif(wareki=="令和"):
            return 2018 + value
            
    

In [114]:
data["builted_year"].unique()

array(['昭和63年', '昭和64年', '平成18年', '昭和62年', '平成14年', '平成7年', '平成13年',
       '平成9年', '平成3年', '平成11年', '平成2年', '平成10年', '平成21年', '昭和59年', '平成8年',
       '平成6年', '平成19年', '平成20年', '平成25年', '平成22年', '平成27年', '昭和61年',
       '平成15年', '平成17年', '平成4年', '平成16年', '昭和60年', '平成24年', '平成26年',
       '昭和52年', '昭和49年', '平成23年', '平成12年', nan, '昭和50年', '昭和54年', '昭和47年',
       '平成31年', '昭和57年', '平成5年', '昭和58年', '昭和55年', '昭和56年', '平成28年',
       '昭和46年', '昭和44年', '平成30年', '昭和53年', '昭和51年', '平成29年', '戦前',
       '昭和36年', '昭和48年', '令和2年', '昭和43年', '令和3年', '昭和23年', '昭和31年',
       '昭和40年', '昭和45年', '昭和41年', '昭和28年', '昭和42年', '昭和22年', '昭和39年',
       '昭和37年', '昭和27年', '昭和33年', '昭和32年', '昭和25年', '昭和24年', '昭和26年',
       '昭和34年', '昭和38年', '昭和21年', '昭和35年', '昭和29年', '昭和30年', '令和4年'],
      dtype=object)

In [117]:
data.builted_year.apply(lambda x:convert_builted_year(x)).dtypes

dtype('float64')

In [88]:
year_process("令和元年")

2019

# floor_type　の前処理

部屋のタイプを大きさ順にしてランキングにして返す

In [92]:
data.floor_type.unique()

array(['２ＬＤＫ', '１Ｋ', '４ＬＤＫ', '３ＤＫ', '３ＬＤＫ', '１ＬＤＫ', '１ＤＫ', nan, '４ＤＫ',
       '２ＤＫ', '２Ｋ', 'オープンフロア', '１Ｒ', '５ＬＤＫ', '１ＬＤＫ＋Ｓ', '３ＬＤＫ＋Ｓ', '１Ｒ＋Ｓ',
       '２ＬＤＫ＋Ｓ', '４Ｋ', '３Ｋ', '３ＤＫ＋Ｓ', '４ＬＤＫ＋Ｓ', '５ＤＫ', '４Ｌ＋Ｋ', 'スタジオ',
       '２ＤＫ＋Ｓ', '６ＤＫ', '６ＬＤＫ＋Ｓ', '７ＬＤＫ', '１ＤＫ＋Ｓ', '６ＬＤＫ', '１Ｋ＋Ｓ', '３Ｄ',
       '１ＬＫ', '５ＬＤＫ＋Ｓ', '７ＤＫ', '３ＬＫ', '５Ｋ', '２Ｋ＋Ｓ', '１Ｌ', '４ＤＫ＋Ｓ', '３ＬＤ',
       '８ＬＤＫ', '２ＬＫ', '３ＬＤＫ＋Ｋ', '３ＬＤ＋Ｓ', '４Ｌ', 'メゾネット', '４ＬＤＫ＋Ｋ',
       '８ＬＤＫ＋Ｓ', '７ＬＤＫ＋Ｓ', '２ＬＤ＋Ｓ', '２Ｌ', '２ＬＤＫ＋Ｋ', '１ＬＤ＋Ｓ', '２ＬＫ＋Ｓ',
       '５ＬＤＫ＋Ｋ', '２Ｌ＋Ｓ', '３Ｋ＋Ｓ', '１ＤＫ＋Ｋ', '２Ｄ', '１ＬＫ＋Ｓ', '１Ｌ＋Ｓ', '２ＬＤ',
       '４Ｄ'], dtype=object)

# area の前処理

object になっているのでfloatにして返す

In [96]:
data.area.unique()

array(['45', '15', '80', '60', '75', '70', '40', '20', '85', '65', '55',
       '25', '95', '30', '90', '120', '35', '105', '500', '50', '100',
       '110', '160', '130', '10', '180', '400', '330', '115', '135',
       '140', '230', '150', '290', '145', '170', '210', '630', '125',
       '155', '200', '190', '220', '2000㎡以上', '260', '480', '360', '195',
       '175', '320', '440', '165', '740', 80, 20, 75, 70, 65, 25, 10, 100,
       15, 85, 60, 45, 35, 50, 140, 55, 30, 110, 105, 95, 40, 90, 115,
       135, 120, 150, 145, 280, 165, 370, 190, 130, 350, 125, 160, 195,
       '830', '470', '300', '980', '710', '270', '820', 460, 520, 180,
       230, 310, 260, 570, 175, 480, 155, 300, 185, 270, 170, 200, 680,
       390, 410, 220, 790, 240, 600, 250, 730, 340, 690, 650, 760, 670,
       290, 970, 440, 610, '590', '780', '650', '185', '410', '660',
       '250', '620', '240', '760', '670', '1300', '340', '460', 210, 500,
       1100, 700, 450, 490, 800, 430, 950, 1000, 510, 320, 550, 360

In [121]:
def convert_area(area):
    if (area=="2000㎡以上"):
        return 2000
    else:
        return int(area)

In [103]:
data["area"].apply(lambda x :area_convert(x)).dtypes

dtype('int64')

# structure の前処理

構造の定義がよくわからないので調べる

# sold_year の前処理

"builted_year"のように西暦にして返す

・もしかしたら「上半期」　「下半期」を取り出して別の変数にして扱うかも

In [104]:
data.sold_year.unique()

array(['2021年第2四半期', '2020年第2四半期', '2020年第4四半期', '2020年第1四半期',
       '2007年第3四半期', '2013年第4四半期', '2015年第1四半期', '2006年第2四半期',
       '2010年第1四半期', '2011年第1四半期', '2006年第4四半期', '2016年第3四半期',
       '2012年第4四半期', '2010年第3四半期', '2008年第3四半期', '2019年第1四半期',
       '2015年第2四半期', '2016年第2四半期', '2019年第2四半期', '2013年第2四半期',
       '2011年第4四半期', '2018年第3四半期', '2018年第1四半期', '2007年第4四半期',
       '2013年第3四半期', '2014年第3四半期', '2020年第3四半期', '2017年第4四半期',
       '2019年第3四半期', '2012年第3四半期', '2017年第3四半期', '2010年第4四半期',
       '2012年第2四半期', '2021年第1四半期', '2009年第4四半期', '2017年第2四半期',
       '2008年第4四半期', '2009年第2四半期', '2016年第4四半期', '2013年第1四半期',
       '2015年第4四半期', '2007年第1四半期', '2018年第4四半期', '2016年第1四半期',
       '2011年第2四半期', '2014年第2四半期', '2015年第3四半期', '2007年第2四半期',
       '2019年第4四半期', '2006年第3四半期', '2014年第4四半期', '2010年第2四半期',
       '2012年第1四半期', '2018年第2四半期', '2009年第1四半期', '2011年第3四半期',
       '2008年第2四半期', '2009年第3四半期', '2014年第1四半期', '2017年第1四半期',
       '2008年第1四半期', '2006年第1四半期', '2005年第3四半期', '2005年

In [105]:
def convert_sold_year(year):
    return int(year[:4])
    

In [108]:
data.sold_year.apply(lambda x:convert_sold_year(x)).dtypes

dtype('int64')

In [91]:
data.dtypes

nearest_station    float64
floor_type          object
area                object
builted_year        object
structure           object
coverrage          float64
floor_rate         float64
sold_year           object
改装                  object
取引価格（総額）_log       float64
dtype: object

data.head

In [89]:
data.head()

,nearest_station,floor_type,area,builted_year,structure,coverrage,floor_rate,sold_year,改装,取引価格（総額）_log
0,7.0,２ＬＤＫ,45,昭和63年,ＲＣ,60.0,150.0,2021年第2四半期,改装済,7.041393
1,4.0,１Ｋ,15,昭和64年,ＳＲＣ,80.0,400.0,2020年第2四半期,未改装,6.602060
2,18.0,４ＬＤＫ,80,昭和63年,ＲＣ,60.0,150.0,2020年第4四半期,改装済,7.397940
3,5.0,２ＬＤＫ,60,平成18年,ＲＣ,80.0,400.0,2020年第1四半期,未改装,7.278754
4,6.0,３ＤＫ,60,昭和62年,ＳＲＣ,80.0,400.0,2007年第3四半期,未改装,6.929419


# 処理のまとめ

In [127]:
def data_processing(data:pd.DataFrame)->pd.DataFrame:
    df = data.copy()
    df.nearest_station = df.nearest_station.apply(lambda x: convert_nearest_station(x))
    df.builted_year = df.builted_year.apply(lambda x:convert_builted_year(x))
    df.area = df.area.apply(lambda x:convert_area(x))
    df.sold_year = df.sold_year.apply(lambda x:convert_sold_year(x))
    
    dummies = pd.get_dummies(data[['改装']])
    df = pd.concat([df.drop('改装',axis=1),dummies[['改装_改装済']]],axis = 1)
    
    df = df.drop(["floor_type","structure"],axis=1)
    return df
    
    

In [128]:
processed_data = data_processing(data)
processed_data

,nearest_station,area,builted_year,coverrage,floor_rate,sold_year,取引価格（総額）_log,改装_改装済
0,7.0,45,1988.0,60.0,150.0,2021,7.041393,1
1,4.0,15,1989.0,80.0,400.0,2020,6.602060,0
2,18.0,80,1988.0,60.0,150.0,2020,7.397940,1
3,5.0,60,2006.0,80.0,400.0,2020,7.278754,0
4,6.0,60,1987.0,80.0,400.0,2007,6.929419,0
...,...,...,...,...,...,...,...,...
37374,NaN,70,1980.0,NaN,NaN,2009,7.342423,1
37375,16.0,70,2003.0,60.0,200.0,2012,7.230449,0
37376,6.0,80,2006.0,60.0,200.0,2014,7.380211,0
37377,45.0,45,1968.0,NaN,NaN,2012,6.770852,0
